In [ ]:
import cv2   # OpenCV library for computer vision

# Load pre-trained Haar Cascade model for face detection
face = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

# Load pre-trained Haar Cascade model for eye detection
eye = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_eye.xml"
)

# Open default webcam (0 = primary camera)
cap = cv2.VideoCapture(0)

# Infinite loop for real-time video stream
while True:
    # Read one frame from webcam
    ret, frame = cap.read()
    if not ret:
        break   # Exit if frame is not captured

    # Convert frame to grayscale and apply histogram equalization
    # This improves contrast for better detection
    gray = cv2.equalizeHist(
        cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    )

    # Detect faces in the grayscale image
    faces = face.detectMultiScale(
        gray, 
        scaleFactor=1.1, 
        minNeighbors=6, 
        minSize=(60, 60)
    )

    # Loop through all detected faces
    for (x, y, w, h) in faces:
        # Draw blue rectangle around face
        cv2.rectangle(
            frame, 
            (x, y), 
            (x+w, y+h), 
            (255, 0, 0), 
            2
        )

        # Extract face region for eye detection
        roi = gray[y:y+h, x:x+w]

        # Detect eyes inside the face region
        eyes = eye.detectMultiScale(
            roi, 
            scaleFactor=1.1, 
            minNeighbors=8, 
            minSize=(20, 20)
        )

        # Loop through detected eyes
        for (ex, ey, ew, eh) in eyes:
            # Only draw eyes in upper half of face
            if ey < h // 2:
                cv2.rectangle(
                    frame,
                    (x+ex, y+ey),
                    (x+ex+ew, y+ey+eh),
                    (0, 255, 0),  # Green box for eyes
                    1
                )

    # Display the output frame
    cv2.imshow("Face & Eye Detection", frame)

    # Exit loop when user presses 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release camera resource
cap.release()

# Close all OpenCV windows
cv2.destroyAllWindows()